In [ ]:
import random


class Card():
    def __init__(self, suit, value):
        '''Сперва определим что такое карта - это вывод значения и масти'''

        self.suit = suit    
        self.value = value


    def __str__(self):
        '''Вернем строку из значения карты'''
        
        return " ".join((self.value, self.suit))

In [ ]:
class Deck:
        def __init__(self):
            '''Определим что такое колода карт - это всевозможные комбинации мастей и значений'''

            self.cards = [Card(s, v) for s in ["Крести", "Пики", "Черви",
                        "Бубен"] for v in ["2", "3", "4", "5", "6", 
                        "7", "8", "9", "10", "В", "Д", "К", "Т"]]

        def shuffle(self):
            '''Перемешаем карты, если в колоде больше, чем одна карта'''
            
            if len(self.cards) > 1:
                random.shuffle(self.cards)
        
        def deal(self):
            '''При раздаче карт из колоды не может быть повторных карт'''

            if len(self.cards)>1:
                return self.cards.pop(0)


In [ ]:
class Hand():

    def __init__(self, dealer=False):
        '''Определим, что такое рука - это карты, которые раздали игроку.
        Первоначально у игрока нет карт'''
        '''При этом надо учесть особенности раздачи руки дилеру - а именно:
        дилер получает одну карту лицом вниз'''

        self.dealer = dealer
        self.cards = []
        self.value = 0

    def add_card(self, card):
        """Функция добавления карты в руку"""
        self.cards.append(card)

    def calculate_value(self):
        '''Посчитаем значение в руке - если выпала карта-картинка, то значение равно 10.
        Также надо учесть двойную роль Туза - при значении карт больше 21
        Туз меняет свое значение с 11 на 1'''
        self.value = 0
        has_ace = False
        for card in self.cards:
            if card.value.isnumeric():
                self.value += int(card.value)
            else:
                if card.value == "Т":
                    has_ace = True
                    self.value += 11
                else:
                    self.value += 10

        if has_ace and self.value > 21:
            self.value -= 10

    def get_value(self):
        '''Получаем значение руки'''
        self.calculate_value()
        return self.value

    def display(self):
        '''При раздаче дилеру - показываем только вторую карту'''
        if self.dealer:
            print('Скрытая карта')
            print(self.cards[1])
        else:
            for card in self.cards:
                print(card)
            print('Сумма:', self.get_value())

In [ ]:
class SaveGameResults(object):
    """Реализуем контекстный менеджер, который сохраняет результаты проведенной игры"""
    def __init__(self, filename):
        self.filename = filename


    def __enter__(self):
        self.myFile = open(self.filename, 'w')
        return self.myFile

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.myFile:
            self.myFile.close()

In [ ]:
class Game:
    '''Let the Hunger Game begin'''

    def __init__(self):
        pass

    def money_count(money):
        print(money)


    def player_lost(self):
        """Метод проверяет, не набрал ли игрок больше 21"""
        return self.player_hand.get_value() > 21


    def show_winner(self, money):
        """Метод показывает счет игры и определяет победителя при разных условиях"""
        player_hand_value = self.player_hand.get_value()
        dealer_hand_value = self.dealer_hand.get_value()

        print("Итоги")
        print("Ваша рука:", player_hand_value)
        print("Рука дилера:", dealer_hand_value)

        if player_hand_value > dealer_hand_value:
            who_won = "Вы выиграли!"
            money += 10
        elif player_hand_value == dealer_hand_value:
            who_won = "Ничья!"
        elif dealer_hand_value > 21:
            who_won = "Дилер проиграл, Вы выиграли!"
            money += 10
        else:
            who_won = "Дилер выиграл!"
            money -= 10
        
        print(who_won)
        print(f'На вашем счету теперь ${money} ')
        # Сохраним результат с помощью контекстного менеджера
        with SaveGameResults("results.txt") as myFile:
            myFile.write(who_won)




    def play(self):
        '''Определим метод, для того, чтобы начать игру'''
        """Этот же метод при значении False выходит из игры"""
        playing = True

        global money
        money = int(input("На вашем счету $"))

        '''Мешаем колоду, раздаем игроку и дилеру карты'''
        while playing:         
            self.deck = Deck()
            self.deck.shuffle()

            self.player_hand = Hand()
            self.dealer_hand = Hand(dealer=True)

            for i in range(2):
                self.player_hand.add_card(self.deck.deal())
                self.dealer_hand.add_card(self.deck.deal())


            print("Ваша рука:")
            self.player_hand.display()
            print()
            print("Рука дилера:")
            self.dealer_hand.display()

            """Сразу проверяем если выпал блэкджек при первоначальной раздаче"""
            if self.player_hand.get_value() == 21:
                money += 10
                print('У Вас блэкджек, Вы выиграли!')
                print(f'На вашем счету теперь ${money} ')
            elif self.dealer_hand.get_value() == 21:
                money -= 10
                print('У дилера блэкджек, он выиграл!')
                print(f'На вашем счету теперь ${money} ')
            elif self.dealer_hand.get_value() == 21 and self.player_hand.get_value() == 21:
                print("У Вас и у дилера блэкджек, ничья")
                print(f'На вашем счету теперь ${money} ')
            else:
                """Продолжаем играть"""
                game_over = False
    
            """Если не выпал блэкджек, значит игра продолжается, можно добирать карты"""
            while not game_over:
                choice = input("Пожалуйста выберите + / - / stop\n")
                while choice not in ["+", "-", "stop"]:
                    """Пользовательский Exception  при вводе неверного значения"""
                    raise ValueError

                """Если игрок добирает карты, то сразу отображаем его счет и положение игры"""
                if choice in ["+"]:
                    self.player_hand.add_card(self.deck.deal())
                    self.player_hand.display()
                    if self.player_lost():
                        loser = "Вы проиграли!"
                        money -= 10
                        print(loser)
                        print(f'На вашем счету теперь ${money} ')
                        # Сохраним результат с помощью контекстного менеджера
                        with SaveGameResults("results.txt") as myFile:
                            myFile.write(loser)
                        game_over = True

                elif choice in "STOP".lower():
                    print(f'Игра окончена со счетом ${money} ')
                    game_over = True
                    playing = False
                    
                        
                else:
                    """Применяем правило, что дилер не имеет право останавливаться, пока у него не будет 17"""
                    while self.dealer_hand.get_value() < 17:
                        self.dealer_hand.add_card(self.deck.deal())
                    self.show_winner(money)
                    game_over = True
                    


In [ ]:
if __name__ == "__main__":
    '''Запустим игру'''
    game = Game()
    game.play()

На вашем счету $100
Ваша рука:
Т Крести
Т Черви
Сумма: 12

Рука дилера:
Скрытая карта
2 Черви
Пожалуйста выберите + / - / stop
+
Т Крести
Т Черви
К Пики
Сумма: 22
Вы проиграли!
На вашем счету теперь $90 
Ваша рука:
2 Пики
К Черви
Сумма: 12

Рука дилера:
Скрытая карта
8 Пики
Пожалуйста выберите + / - / stop
+
2 Пики
К Черви
В Черви
Сумма: 22
Вы проиграли!
На вашем счету теперь $80 
Ваша рука:
Т Крести
5 Пики
Сумма: 16

Рука дилера:
Скрытая карта
В Пики
Пожалуйста выберите + / - / stop
+
Т Крести
5 Пики
Т Черви
Сумма: 17
Пожалуйста выберите + / - / stop
-
Итоги
Ваша рука: 17
Рука дилера: 20
Дилер выиграл!
На вашем счету теперь $70 
Ваша рука:
8 Бубен
К Крести
Сумма: 18

Рука дилера:
Скрытая карта
Т Крести
У дилера блэкджек, он выиграл!
На вашем счету теперь $70 
Ваша рука:
8 Крести
7 Крести
Сумма: 15

Рука дилера:
Скрытая карта
К Черви
Пожалуйста выберите + / - / stop
+
8 Крести
7 Крести
Д Черви
Сумма: 25
Вы проиграли!
На вашем счету теперь $60 
Ваша рука:
В Бубен
Д Черви
Сумма: 20

Рука